# Setup

In [1]:

import os
import openai
import requests
 

client = openai.OpenAI(api_key=input())
os.environ["OPENAI_API_KEY"] = input() 
def llm(prompt, stop=["\n"]):
    response = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=[{'role' : 'user', 'content' : prompt}],
      temperature=0,
      max_tokens=100,
      top_p=1,
      frequency_penalty=0.0,
      presence_penalty=0.0,
      stop=stop
    )
    return response.choices[0].message.content

In [2]:
%pip install gym

Note: you may need to restart the kernel to use updated packages.


In [3]:
import scraper, wrappers
env = scraper.WikiSearch(api_key=input())
env = wrappers.HotPotQAWrapper(env, split="dev")
env = wrappers.LoggingWrapper(env)

def step(env, action):
    attempts = 0
    while attempts < 10:
        try:
            return env.step(action)
        except requests.exceptions.Timeout:
            attempts += 1

# ReAct

In [4]:
import json
import sys

folder = './prompts/'
prompt_file = 'prompts_naive.json'
with open(folder + prompt_file, 'r') as f:
    prompt_dict = json.load(f)

# webthink_examples = prompt_dict['webthink_simple6']
# instruction = """Solve a question answering task with interleaving Thought, Action, Observation steps. Thought can reason about the current situation, and Action can be three types: 
# (1) Search[entity], which searches the exact entity on Wikipedia and returns the first paragraph if it exists. If not, it will return some similar entities to search.
# (2) Lookup[keyword], which returns the next sentence containing keyword in the current passage.
# (3) Finish[answer], which returns the answer and finishes the task.
# Here are some examples.
# """
# webthink_prompt = instruction + webthink_examples
with open('prompts/team.txt', 'r') as f:
    prompt = '\n'.join(f.readlines())
def webthink(idx=None, prompt=prompt, to_print=True):
    question = env.reset(idx=idx)
    if to_print:
        print(idx, question)
    prompt += question + "\n"
    n_calls, n_badcalls = 0, 0
    for i in range(1, 8):
        n_calls += 1
        try:
            thought_action = llm(prompt + f"Thought {i}:", stop=[f"\nObservation {i}:"])
        except Exception as e:
            print(f'too long? bad error {e}')
            return 0, info
        try:
            thought, action = thought_action.strip().split(f"\nAction {i}: ")
        except:
            print('ohh...', thought_action)
            n_badcalls += 1
            n_calls += 1
            thought = thought_action.strip().split('\n')[0]
            action = llm(prompt + f"Thought {i}: {thought}\nAction {i}:", stop=[f"\n"]).strip()
        obs, r, done, info = step(env, action[0].lower() + action[1:])
        obs = obs.replace('\\n', '')
        step_str = f"Thought {i}: {thought}\nAction {i}: {action}\nObservation {i}: {obs}\n"
        prompt += step_str
        if to_print:
            print(step_str)
        if done:
            break
    if not done:
        obs, r, done, info = step(env, "finish | NO ANSWER REACHED")
    if to_print:
        print(info, '\n')
    info.update({'n_calls': n_calls, 'n_badcalls': n_badcalls, 'traj': prompt})
    return r, info

In [6]:
import random
import time
from tqdm import tqdm
import json

idxs = list(range(7405))
# random.Random(233).shuffle(idxs)

rs = []
infos = []
old_time = time.time()
ID = 11
try:
# for i in tqdm(idxs[ID:ID+1]):
    for i in tqdm(idxs[600:1000]):
        print(f'INDEX = {i}')
        r, info = webthink(i, to_print=False)
        rs.append(info['em'])
        infos.append(info)
        print(sum(rs), len(rs), sum(rs) / len(rs), (time.time() - old_time) / len(rs))
        print('-----------')
        print()
except Exception as e:
    print(e)
    print('Died for unknown reason')

with open('151-react_baseline_2.json', 'w') as f:
    json.dump(infos, f)

  0%|          | 0/929 [00:00<?, ?it/s]

['Background']
['Background']
No results found.
Unknown error occured during finding and scraping...
['Background']


  0%|          | 1/929 [00:22<5:51:09, 22.70s/it]

OURS: no answer reached
CORRECT: 276170 inhabitants
0 1 0.0 22.705573797225952
-----------

['History', 'Importance and meaning']
['Festivities', 'Activities']


  0%|          | 2/929 [00:31<3:48:22, 14.78s/it]

...

OURS: beatles
CORRECT: beatles
26 78 0.3333333333333333 18.131700680806087
-----------

['Weekly charts', 'Year-end charts', 'Commercial performance']


  9%|▊         | 79/929 [23:41<4:07:58, 17.50s/it]

OURS: chattahoochee
CORRECT: well burn that bridge
26 79 0.3291139240506329 17.987356201002868
-----------

['Television and film career (1988–2005)', 'Harry Potter (2006–2011)', 'Tyrant and Tarzan (2012–2016)', 'Fantastic Beasts series (2016–2022)', 'Pain Hustlers and current work (2023–present)']
['Director', 'Writer', 'Actor']
['A']


  9%|▊         | 79/929 [24:20<4:21:57, 18.49s/it]

Error code: 400 - {'error': {'message': "This model's maximum context length is 16385 tokens. However, your messages resulted in 21560 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}
Died for unknown reason
